In [1]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import os
import glob

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
faults_diagnostics = pd.read_csv('faults_diagnostics.csv')

In [4]:
faults_diagnostics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1054113 entries, 0 to 1054112
Data columns (total 43 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Unnamed: 0                 1054113 non-null  int64  
 1   FaultId                    1054113 non-null  int64  
 2   AcceleratorPedal           1054113 non-null  float64
 3   BarometricPressure         1054113 non-null  float64
 4   CruiseControlActive        1054113 non-null  object 
 5   CruiseControlSetSpeed      520327 non-null   float64
 6   DistanceLtd                1054113 non-null  float64
 7   EngineCoolantTemperature   1054113 non-null  float64
 8   EngineLoad                 1054113 non-null  float64
 9   EngineOilPressure          1054113 non-null  float64
 10  EngineOilTemperature       1054113 non-null  float64
 11  EngineRpm                  1054113 non-null  float64
 12  EngineTimeLtd              1054113 non-null  float64
 13  FuelLevel   

In [3]:
faults_diagnostics.drop('Unnamed: 0', axis=1)

,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,...,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,0.0,14.2100,False,66.48672,423178.70000,100.4,11.0,0.00,96.74375,...,0,111,17,True,2.0,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,9.2,14.3550,unknown,NaN,368419.30000,181.4,18.0,36.54,201.08750,...,11,629,12,True,127.0,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,9.2,14.3550,unknown,NaN,368419.30000,181.4,18.0,36.54,201.08750,...,11,1807,2,False,127.0,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,9.2,14.3550,unknown,NaN,368419.30000,181.4,18.0,36.54,201.08750,...,11,1807,2,True,127.0,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,17.6,3.5525,unknown,NaN,366722.90000,177.8,28.5,45.82,225.78130,...,0,4364,17,False,2.0,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054108,1248454,25.2,14.3550,unknown,NaN,369586.50000,185.0,28.0,36.54,208.45630,...,0,1761,17,False,3.0,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21.000
1054109,1248455,100.0,14.5000,True,64.62260,423937.90000,185.0,51.0,37.12,211.49370,...,0,1569,31,True,5.0,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59.000
1054110,1248456,0.0,14.3550,True,66.48672,465925.40000,186.8,62.0,41.18,212.84380,...,0,3216,10,True,1.0,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14.000
1054111,1248457,1.6,14.4275,False,67.72946,28606.65625,181.4,0.0,27.26,221.73120,...,0,111,18,True,8.0,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000


In [5]:
faults_diagnostics['EventTimeStamp'] = pd.to_datetime(faults_diagnostics['EventTimeStamp'])

In [6]:
faults_diagnostics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1054113 entries, 0 to 1054112
Data columns (total 43 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   Unnamed: 0                 1054113 non-null  int64         
 1   FaultId                    1054113 non-null  int64         
 2   AcceleratorPedal           1054113 non-null  float64       
 3   BarometricPressure         1054113 non-null  float64       
 4   CruiseControlActive        1054113 non-null  object        
 5   CruiseControlSetSpeed      520327 non-null   float64       
 6   DistanceLtd                1054113 non-null  float64       
 7   EngineCoolantTemperature   1054113 non-null  float64       
 8   EngineLoad                 1054113 non-null  float64       
 9   EngineOilPressure          1054113 non-null  float64       
 10  EngineOilTemperature       1054113 non-null  float64       
 11  EngineRpm                  1054113 no

In [ ]:
(faults_diagnostics['EventTimeStamp'] - faults_diagnostics['EventTimeStamp'].min()).dt.days + 1

In [ ]:
faults_diagnostics.drop_duplicates(subset=['EquipmentID', 'spn', 'fmi', 'EventTimeStamp'], keep='first', inplace=True)

In [ ]:
faults_diagnostics.head()